In [10]:
from pathlib import Path

def find_fire_detection_dir():
    path = Path.cwd()
    if path.name == "MachineLearning":
        return path
    for parent in path.parents:
        if parent.name == "MachineLearning":
            return parent
    raise Exception("MachineLearning directory not found")


project_root = find_fire_detection_dir()
src_folder_path = project_root / "src"
weights_folder_path = project_root / "weights"
test_data_folder_path = project_root / "test_data"

In [11]:
# A scrip that moves each best model from a specified folder to the folder of weights, then run the test and save the results in csv files
import os
import sys
import shutil
import subprocess
import argparse
import glob
from pathlib import Path
import pandas as pd


model = "x3d_xs.pth"
experiments_name = ["x3d_xs 3-trained Cristian", "x3d_xs 4-trained Cristian", "x3d_xs all-trained Cristian", "x3d_xs 5-trained Grazia"]
test_command = "python " + str(src_folder_path / "test.py")  + " --videos ../test_data/TEST_SET/ --results ../test_data/TEST_RESULTS/ --ground_truth ../test_data/GT_TEST_SET/ --model x3d_xs --clip_len 13 --clip_stride 13"

In [12]:
from tqdm import tqdm

for experiment_name in tqdm(experiments_name):
    best_model_folder = src_folder_path / "experiments" / experiment_name
    
    if not best_model_folder.exists():
        print("The folder {} doesn't exist".format(best_model_folder))
        continue

    # Get all the files in best_model_folder that end with .pth and are called best_model, sort them
    best_model_files = glob.glob(str(best_model_folder) + "/best_model*.pth")


    best_model_files.sort(key=lambda x: int(x.split("_")[-1].split(".")[0]))

    
    # Create pandas dataframe to store the results, the columns are the metrics
    df = pd.DataFrame(columns=["best_model", "precision", "recall", "f-score", "and", "nand", "pfr", "mem","fds"])

    # Delete the results folder and create a new one
    shutil.rmtree(test_data_folder_path / "TEST_RESULTS")
    os.mkdir(test_data_folder_path / "TEST_RESULTS")

    for i,best_model in enumerate(best_model_files):
        best_model = Path(best_model)
        print("Testing model: {}/{}".format(best_model.parent.name, best_model.name))
        # Copy the best_model file in the weights folder and rename it as model
        shutil.copy(best_model, weights_folder_path / model)

        # Run the test and collect the output
        output = subprocess.run(test_command.split(" "), capture_output=True)

        # Get the metrics from the output and add them to the dataframe
        metrics = pd.read_csv("../test_data/TEST_RESULTS/metrics.csv")
        df.loc[i] = [best_model.name, metrics["precision"][0], metrics["recall"][0], metrics["f-score"][0], metrics["and"][0], metrics["nand"][0], metrics["pfr"][0], metrics["mem"][0], metrics["fds"][0]]
        # Delete the model from the weights folder
        os.remove(weights_folder_path / model)

        # Delete the results folder and create a new one
        shutil.rmtree(test_data_folder_path / "TEST_RESULTS")
        os.mkdir(test_data_folder_path / "TEST_RESULTS")

    
    # Save the dataframe in a csv file
    df.to_csv(Path.cwd() / "{}.csv".format(experiment_name))

  0%|          | 0/4 [00:00<?, ?it/s]

Testing model: best_model_0.pth
